In [1]:
# Not needed if tinc-python is installed
import sys
sys.path.append('../external/tinc/tinc-python/tinc-python')
from tinc_client import *
tclient = TincClient()

Connected to localhost:34450. Server version 1 revision 0


In [2]:
#tclient.synchronize()

from parameter import *

eci1_param=tclient.create_parameter(Parameter,\
                                    "tet_oct_eci","casm",-0.375,0.375,\
                                    [-0.375,-0.125,0.125,0.375],-0.375)

eci2_param=tclient.create_parameter(Parameter,\
                                    "oct_tet_NN","casm",0.0,6.0,[0.0,0.25,0.5,0.75,1.0,1.25,1.5,1.75,2.0,6.0],6.0)

eci3_param=tclient.create_parameter(Parameter,\
                                    "oct_oct_NN","casm",0.0,1.0,\
                                    [0.0,1.0],0.0)

eci4_param=tclient.create_parameter(Parameter,\
                                    "tet_tet_NN","casm",0.0,0.5,[0.0,0.5],0.0)

In [3]:
root_dir = "D:/data_for_visualization/visualization/"
#root_dir = "C:/Users/Andres/source/repos/vdv_data/visualization/"
import shutil
def create_dir_string_from_eci_param(eci_value):
    prefix= "AMX2_spinel_diffusion_0.0_0.0"+ \
    "_"+str(eci1_param.value)+\
    "_"+str(eci2_param.value)+\
    "_"+str(eci3_param.value)+\
    "_"+str(eci4_param.value)
    datasetname=root_dir +prefix+\
    "/kinetic_mc"
    tclient.get_parameter("dataset").value=datasetname
    voltage_buffer=tclient.get_disk_buffer('graph1')
    voltage_buffer.enable_round_robin(5)
    fname=voltage_buffer.get_filename_for_writing()
    print(fname)
    shutil.copyfile("C:/Users/askje/OneDrive/Documents/flow_chart_pngs/"+prefix+"flow_chart_voltage.png",fname)
    voltage_buffer.done_writing_file(fname)
    diffusion_buffer=tclient.get_disk_buffer('graph2')
    diffusion_buffer.enable_round_robin(5)
    fname=diffusion_buffer.get_filename_for_writing()
    shutil.copyfile("C:/Users/askje/OneDrive/Documents/flow_chart_pngs/"+prefix+"flow_chart_diffusion.png",fname)
    diffusion_buffer.done_writing_file(fname)
    corr_fact_buffer=tclient.get_disk_buffer('graph3')
    corr_fact_buffer.enable_round_robin(5)
    fname=corr_fact_buffer.get_filename_for_writing()
    shutil.copyfile("C:/Users/askje/OneDrive/Documents/flow_chart_pngs/"+prefix+"flow_chart_corrfact.png",fname)
    corr_fact_buffer.done_writing_file(fname)
    return

In [4]:
eci1_param.register_callback(create_dir_string_from_eci_param)

eci2_param.register_callback(create_dir_string_from_eci_param)

eci3_param.register_callback(create_dir_string_from_eci_param)

eci4_param.register_callback(create_dir_string_from_eci_param)

In [5]:
eci1_param.value=0.125
eci2_param.value=6.0
eci3_param.value=1.0
eci4_param.value=0.0
tclient.wait_for_server_available()

C:\Users\askje\ARG_codes\casm_viewer\bin\cachedGraph\currentGraph1_0.png
C:\Users\askje\ARG_codes\casm_viewer\bin\cachedGraph\currentGraph1_1.png
C:\Users\askje\ARG_codes\casm_viewer\bin\cachedGraph\currentGraph1_2.png
C:\Users\askje\ARG_codes\casm_viewer\bin\cachedGraph\currentGraph1_3.png


In [6]:
import random
import matplotlib.pyplot as plt
import threading
import shutil
imageBuffer = tclient.get_disk_buffer('graph')
imageBuffer.enable_round_robin(5)
image_names=["oct_doub_va_cleared_path",
            "oct_doub_va_double_cleared_path",
             "oct_doub_va_queued_path",
             "oct_no_va",
             "oct_sing_va",
             "oct_trip_va_forced_path",
             "oct_trip_va_free_range_path",
             "oct_trip_va_semiforced_path",
             "oct_trip_va_trapped_path",
             "tet_no_va",
             "tet_sing_va"]
def update_shellsite_pic(shellSite):
    print("Shell bitstring " + str(shellSite))
    b = int(shellSite)
    current_image_name=""
    for iname in image_names:
        if b & 1 == 1:
            current_image_name=iname
            break
        b = b >> 1
    fname = imageBuffer.get_filename_for_writing()
    print(fname)
    shutil.copyfile("C:/Users/askje/OneDrive/Documents/"+current_image_name+".png",fname)
    imageBuffer.done_writing_file(fname)

    print(current_image_name)

shellsites = tclient.get_parameter("ShellSiteTypes")
shellsites.register_callback(update_shellsite_pic,False)
shellsites.maximum=2048

In [7]:
param=tclient.parameter_spaces[0]
root_path=param.get_root_path()
trajectories_path=root_path+param.get_current_relative_path()+"\\trajectory.nc"
print(trajectories_path)

D:\data_for_visualization\visualization\AMX2_spinel_diffusion_0.0_0.0_0.125_6.0_1.0_0.0\kinetic_mc\0\conditions.0\\trajectory.nc


In [8]:
import xarray as xr
import numpy as np

def step_generator():
    root_path=param.get_root_path()
    trajectories_path=root_path+param.get_current_relative_path()+"\\trajectory.nc"
    print(f'processing: {trajectories_path}')
    try:
        chunkedby10ds=xr.open_dataset(trajectories_path,chunks={'sample_dim':100})
    except FileNotFoundError:
        print(f"file not found: {trajectories_path}")
        return []
    #choose an atom to track
    #find the first step that site index changes, record new position
    #track that position from then on
    # repeat if changes. 
    previous_dof=np.array(chunkedby10ds['occupation_dofs'][0][:].values)
    atom_locs=np.where(previous_dof==1)[0].tolist()
    atom_to_track=atom_locs
    steps_changed=[[] for i in range(len(atom_locs))]
    swap_pairs=[[] for i in range(3001)]
    site_labels=[[atom_locs[i]] for i in range(len(atom_locs))]
    previous_dof=np.array(chunkedby10ds['occupation_dofs'][0][:].values)
    for step_count in range(1,3001):
        new_dof=np.array(chunkedby10ds['occupation_dofs'][step_count][:].values)
        diff_dof=previous_dof-new_dof
        #where diff_dof is 1 this is the site index where atom 
        #disappeared 1->0 
        disap_ix=np.where(diff_dof==1)[0][0]
        track_ind=atom_to_track.index(disap_ix)
        #time value is 1 more than step count
        steps_changed[track_ind].append(step_count+1)
        #255 means -1 this is the site index where atom 
        #appeared
        ap_ix=np.where(diff_dof==255)[0][0]
        atom_to_track[track_ind]=ap_ix
        swap_pairs[step_count]=[int(disap_ix),int(ap_ix)]
        site_labels[track_ind].append(atom_to_track[track_ind])
        previous_dof=new_dof
    print("steps generated")
    return (steps_changed,swap_pairs)

In [9]:
#tclient.synchronize()
ps = tclient.get_parameter_space("casmParameters")
ps.enable_cache()

starting async callback <function update_shellsite_pic at 0x000001E165236168>
Shell bitstring 0
starting async callback <function update_shellsite_pic at 0x000001E165236168>C:\Users\askje\ARG_codes\casm_viewer\bin\currentGraph_0.png

Shell bitstring 0
C:\Users\askje\ARG_codes\casm_viewer\bin\currentGraph_1.png
Exception in *async* parameter callback (Continuing):Exception in *async* parameter callback (Continuing):

starting async callback <function update_shellsite_pic at 0x000001E165236168>
Shell bitstring 0
C:\Users\askje\ARG_codes\casm_viewer\bin\currentGraph_2.png
Exception in *async* parameter callback (Continuing):


Traceback (most recent call last):
  File "../external/tinc/tinc-python/tinc-python\parameter.py", line 299, in _cb_async_wrapper
    cb(value)
  File "<ipython-input-6-d59d097cb7ca>", line 29, in update_shellsite_pic
    shutil.copyfile("C:/Users/askje/OneDrive/Documents/"+current_image_name+".png",fname)
  File "C:\Users\askje\anaconda3\lib\shutil.py", line 120, in copyfile
    with open(src, 'rb') as fsrc:
FileNotFoundError: [Errno 2] No such file or directory: 'C:/Users/askje/OneDrive/Documents/.png'
Traceback (most recent call last):
Traceback (most recent call last):
  File "../external/tinc/tinc-python/tinc-python\parameter.py", line 299, in _cb_async_wrapper
    cb(value)
  File "<ipython-input-6-d59d097cb7ca>", line 29, in update_shellsite_pic
    shutil.copyfile("C:/Users/askje/OneDrive/Documents/"+current_image_name+".png",fname)
  File "C:\Users\askje\anaconda3\lib\shutil.py", line 120, in copyfile
    with open(src, 'rb') as fsrc:
FileNotFoundError: [Errno 2] No such file 

starting async callback <function update_shellsite_pic at 0x000001E165236168>
Shell bitstring 0
C:\Users\askje\ARG_codes\casm_viewer\bin\currentGraph_3.png
Exception in *async* parameter callback (Continuing):


Traceback (most recent call last):
  File "../external/tinc/tinc-python/tinc-python\parameter.py", line 299, in _cb_async_wrapper
    cb(value)
  File "<ipython-input-6-d59d097cb7ca>", line 29, in update_shellsite_pic
    shutil.copyfile("C:/Users/askje/OneDrive/Documents/"+current_image_name+".png",fname)
  File "C:\Users\askje\anaconda3\lib\shutil.py", line 120, in copyfile
    with open(src, 'rb') as fsrc:
FileNotFoundError: [Errno 2] No such file or directory: 'C:/Users/askje/OneDrive/Documents/.png'


starting async callback <function update_shellsite_pic at 0x000001E165236168>
Shell bitstring 0
C:\Users\askje\ARG_codes\casm_viewer\bin\currentGraph_4.png
starting async callback <function update_shellsite_pic at 0x000001E165236168>
Shell bitstring 0
C:\Users\askje\ARG_codes\casm_viewer\bin\currentGraph_0.png
Exception in *async* parameter callback (Continuing):
Exception in *async* parameter callback (Continuing):
starting async callback <function update_shellsite_pic at 0x000001E165236168>
Shell bitstring 1024
C:\Users\askje\ARG_codes\casm_viewer\bin\currentGraph_1.png
starting async callback <function update_shellsite_pic at 0x000001E165236168>
Shell bitstring 1024
C:\Users\askje\ARG_codes\casm_viewer\bin\currentGraph_2.png
tet_sing_va
tet_sing_va


Traceback (most recent call last):
  File "../external/tinc/tinc-python/tinc-python\parameter.py", line 299, in _cb_async_wrapper
    cb(value)
  File "<ipython-input-6-d59d097cb7ca>", line 29, in update_shellsite_pic
    shutil.copyfile("C:/Users/askje/OneDrive/Documents/"+current_image_name+".png",fname)
  File "C:\Users\askje\anaconda3\lib\shutil.py", line 120, in copyfile
    with open(src, 'rb') as fsrc:
FileNotFoundError: [Errno 2] No such file or directory: 'C:/Users/askje/OneDrive/Documents/.png'
Traceback (most recent call last):
  File "../external/tinc/tinc-python/tinc-python\parameter.py", line 299, in _cb_async_wrapper
    cb(value)
  File "<ipython-input-6-d59d097cb7ca>", line 29, in update_shellsite_pic
    shutil.copyfile("C:/Users/askje/OneDrive/Documents/"+current_image_name+".png",fname)
  File "C:\Users\askje\anaconda3\lib\shutil.py", line 120, in copyfile
    with open(src, 'rb') as fsrc:
FileNotFoundError: [Errno 2] No such file or directory: 'C:/Users/askje/OneDr

In [ ]:
ps.sweep(step_generator,params=[ps.get_parameter("dir")],force_values=True,dependencies=[eci1_param, eci2_param,eci3_param,eci4_param,ps.get_parameter('dir')])

In [10]:
[steps_changed,swap_pairs] = ps.run_process(step_generator,dependencies=[eci1_param, eci2_param,eci3_param,eci4_param,ps.get_parameter('dir')])

Ignoring parameters: ['time', 'dir', 'T', 'param_chem_pot(a)']. Not used in function


In [11]:
param.get_root_path()
param.get_current_relative_path()

'76\\conditions.0\\'

In [12]:
traj_buffer=tclient.get_disk_buffer('trajectory_buffer')
traj_buffer.enable_round_robin(10)
tclient.get_parameter("width", "trajectory").value = 0.5
tclient.get_parameter("alpha", "trajectory").value =0.6
import time
def write_positions(pos):
    import json
    fname = traj_buffer.get_filename_for_writing()

    print(f"Buffer: {fname}")
    with open(fname, 'w') as f:
        json.dump(pos, f)
        traj_buffer.done_writing_file(fname)

In [13]:
# Test writing to the buffer
#write_positions([[[0,0,0]],[[10,10,10]],[[25,25,25]],[[-15,25,15]]])

In [14]:
def correct_arrow(single_coord_list):
    single_arr=np.array(single_coord_list)
    elementary_hop_dist=2.2
    return -single_arr/np.linalg.norm(single_arr)*elementary_hop_dist

In [15]:
import xarray
template_path=ps.get_root_path() + tclient.get_processor("TemplateGenerator").output_files[0]
template_atoms=xarray.open_dataset(template_path)['atoms_var']
def calc_summed_trajectory(value):
    print("registered click")
    [steps_changed,swap_pairs] = ps.run_process(step_generator,dependencies=[eci1_param, eci2_param,eci3_param,eci4_param,ps.get_parameter('dir')])
    arrows=[[[35,35,35]]]
    for i in range(1,len(swap_pairs)):
        disap,ap=swap_pairs[i]
        first=template_atoms[disap].values.tolist()
        second=template_atoms[ap].values.tolist()
        vec=np.array([second[0]-first[0],second[1]-first[1],second[2]-first[2]])
        color=[0.2,0.8,0.0]
        if np.linalg.norm(vec) > 2.5:
            vec=correct_arrow(vec)
            color=[1,1,1]
        arrow=[vec.tolist()]
        arrows.append(arrow)
    print(np.array(arrows).shape)
    write_positions(arrows)
    print("finished")
    return

sum_traj_button=tclient.create_parameter(Trigger,"summed_trajectory","casm")
sum_traj_button.register_callback(calc_summed_trajectory,False)

In [16]:
time_param=tclient.get_parameter("time")
template_path=ps.get_root_path() + tclient.get_processor("TemplateGenerator").output_files[0]
import xarray
template_atoms=xarray.open_dataset(template_path)['atoms_var']
import time
full_stop=True
def individual_trajectories(v):
    global full_stop
    full_stop=True
    [steps_changed,swap_pairs] = ps.run_process(step_generator,dependencies=[eci1_param, eci2_param,eci3_param,eci4_param,ps.get_parameter('dir')])
    ixes_to_check=[]
    for i in range(len(steps_changed)):
        ixes_to_check.append(len(steps_changed[i]))
    arrows=[]
    for ind,check in enumerate(ixes_to_check):
        if check>0:
            for t in steps_changed[ind]:
                time_param.value=t
                disap,ap=swap_pairs[t-1]
                first=template_atoms[disap].values.tolist()
                second=template_atoms[ap].values.tolist()
                vec=np.array([second[0]-first[0],second[1]-first[1],second[2]-first[2]])
                color=[0.2,0.8,0.0]
                if np.linalg.norm(vec) > 2.5:
                    vec=correct_arrow(vec)
                    color=[1,1,1]
                final=np.array(first)+vec
                arrow=[first,final.tolist(),color]
                arrows.append(arrow)
                write_positions(arrows)
                if not full_stop:
                    return
                time.sleep(0.1)
    return
indiv_traj_button=tclient.create_parameter(Trigger,"individual_trajectory","casm")
indiv_traj_button.register_callback(individual_trajectories,False)

In [17]:
clear_traj=tclient.create_parameter(Trigger,"clear_trajectories","casm")

def clear_traj_buffer(value):
    global full_stop
    full_stop=False
    write_positions([])
clear_traj.register_callback(clear_traj_buffer)

In [18]:
selector_slider=tclient.create_parameter(ParameterInt,"moving_atoms","casm")
selector_slider.set_space_type("INDEX")

def sel_atom(val):
    [steps_changed,swap_pairs] = ps.run_process(step_generator,dependencies=[eci1_param, eci2_param,eci3_param,eci4_param,ps.get_parameter('dir')])
    tclient.get_parameter("time").value= steps_changed[val][0]
    return

tclient.get_parameter("moving_atoms").register_callback(sel_atom)

In [19]:
def reset_atom_selection_slider(value):
    print(f"reset_atom_selection_slider {value}")
    [steps_changed,swap_pairs] = ps.run_process(step_generator,dependencies=[eci1_param, eci2_param,eci3_param,eci4_param,ps.get_parameter('dir')])
    all_possible_starts=[]    
    for ix,val in enumerate(steps_changed):
        if len(val)>0:
            all_possible_starts.append(ix)
    print(all_possible_starts)
    slider = tclient.get_parameter("moving_atoms")
    #slider.maximum=len(all_possible_starts)
    slider.values = [int(i) for i in all_possible_starts]
    
tclient.get_parameter("moving_atoms").register_callback(sel_atom)

reset_button=tclient.create_parameter(Trigger,"reset_moving_atoms","casm")
reset_button.register_callback(reset_atom_selection_slider)

In [20]:
# Force updating of atom selection slider
#reset_atom_selection_slider(0.1)

In [21]:
def calc_traj(value):
    global full_stop
    full_stop = True
    [steps_changed,swap_pairs] = ps.run_process(step_generator,dependencies=[eci1_param, eci2_param,eci3_param,eci4_param,ps.get_parameter('dir')])
    idx=int(tclient.get_parameter("moving_atoms").value)
    arrows=[]
    if len(steps_changed[idx])>0:
        for t in steps_changed[idx]:
            time_param.value=t
            disap,ap=swap_pairs[t-1]
            first=template_atoms[disap].values.tolist()
            second=template_atoms[ap].values.tolist()
            vec=np.array([second[0]-first[0],second[1]-first[1],second[2]-first[2]])
            color=[0.2,0.8,0.0]
            if np.linalg.norm(vec) > 2.5:
                vec=correct_arrow(vec)
                color=[1,1,1]
            final=np.array(first)+vec
            arrow=[first,final.tolist(),color]
            arrows.append(arrow)
            write_positions(arrows)
            
            if not full_stop:
                return
            time.sleep(0.1)
    return
calc_traj_buffer=tclient.create_parameter(Trigger,"calc_single_trajectory","casm")
calc_traj_buffer.register_callback(calc_traj,False)

starting async callback <function calc_summed_trajectory at 0x000001E1687629D8>
registered click
Ignoring parameters: ['time', 'dir', 'T', 'param_chem_pot(a)']. Not used in function
(3001, 1, 3)
Buffer: C:\Users\askje\ARG_codes\casm_viewer\bin\trajectory_0.json
finished
Buffer: C:\Users\askje\ARG_codes\casm_viewer\bin\trajectory_1.json
reset_atom_selection_slider True
Ignoring parameters: ['time', 'dir', 'T', 'param_chem_pot(a)']. Not used in function
[1, 2, 3, 5, 6, 7, 9, 10, 11, 12, 15, 17, 18, 20, 21, 22, 24, 26, 27, 31, 32, 33, 35, 36, 37, 38, 39, 42, 44, 45, 46, 47, 48, 49, 50, 54, 55, 56, 58, 60, 61, 63, 65, 66, 67, 68, 72, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 87, 88, 89, 90, 92, 93, 95, 96, 97, 98, 101, 102, 103, 104, 105, 106, 110, 111, 112, 113, 114, 115, 116, 119, 122, 123, 126, 127, 128, 129, 130, 131, 132, 133, 135, 136, 137, 138, 140, 141, 142, 143, 146, 148, 151, 152, 153, 154, 155, 156, 159, 160, 161, 162, 165, 166, 167, 169, 170, 172, 173, 175, 176, 177, 178,

In [ ]:
def print_time(value):
    print(tclient.get_parameter("time").value)
    return
tclient.get_parameter("time").register_callback(print_time)

In [ ]:
currSel=tclient.get_parameter("currentSelection")
tclient.print()
tclient.synchronize()